In [1]:
from datasets import load_dataset, load_from_disk
import epitran

In [22]:
dataset = load_from_disk("/home/toure215/BERT_phonetic/DATASETS/rap/phonetic_rap_ds_hf")
dataset
print(dataset["train"][0])

{'sentence1': 'hIr\\@z @ stOr\\i @bawt D@ Tr\\i', 'sentence2': 'ju k{nt kVm h{N fOr\\ fr\\i', 'label': 'Tr\\i'}


In [18]:
from functools import lru_cache


epi = epitran.Epitran("eng-Latn")


def transcribe(examples):
    sentences1 = examples["sentence1"]
    sentences2 = examples["sentence2"]
    labels = examples["label"]
    return {
        "sentence1": [epi.transliterate(sentence) for sentence in sentences1],
        "sentence2": [epi.transliterate(sentence) for sentence in sentences2],
        "label": [epi.transliterate(label) for label in labels],
    }


dataset = dataset.map(lambda x: {"Verse": epi.transliterate(x["Verse"])}, num_proc=12)

Map (num_proc=12):   0%|          | 0/161190 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/19901 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/17911 [00:00<?, ? examples/s]

In [19]:
print(dataset["train"][0])

{'Verse': 'hɪz wɪzdəm livz nɑt dɪsɪnɡejd͡ʒd fɹʌm hɛvən', 'label': 0}


In [21]:
dataset.save_to_disk("/home/toure215/BERT_phonetic/DATASETS/verses/verses_hf_IPA")

Saving the dataset (0/1 shards):   0%|          | 0/161190 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/19901 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/17911 [00:00<?, ? examples/s]

In [2]:
bookcorpus = load_from_disk("/home/toure215/BERT_phonetic/DATASETS/bookcorpus")
wikitext = load_from_disk("/home/toure215/BERT_phonetic/DATASETS/wikitext")

print(bookcorpus)
print(wikitext)

Loading dataset from disk:   0%|          | 0/50 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 113588716
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1147194
    })
})
DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4170
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1695602
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3626
    })
})


In [3]:
bookcorpus = bookcorpus.map(
    lambda x: {"num_words": len(x["text"].split())}, num_proc=15
)
wikitext = wikitext.map(lambda x: {"num_words": len(x["text"].split())}, num_proc=15)

In [7]:
print(bookcorpus["train"][:10]["num_words"])

[99, 3, 5, 6, 13, 16, 22, 18, 98, 51]


In [15]:
bookcorpus_acc = [0]
wikitext_acc = [0]


def sum_words(examples, acc):
    acc[0] += examples["num_words"]


bookcorpus.map(lambda x: sum_words(x, bookcorpus_acc))
wikitext.map(lambda x: sum_words(x, wikitext_acc))

Map:   0%|          | 0/113588716 [00:00<?, ? examples/s]

Map:   0%|          | 0/1147194 [00:00<?, ? examples/s]

Map:   0%|          | 0/4170 [00:00<?, ? examples/s]

Map:   0%|          | 0/1695602 [00:00<?, ? examples/s]

Map:   0%|          | 0/3626 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['text', 'num_words'],
        num_rows: 4170
    })
    train: Dataset({
        features: ['text', 'num_words'],
        num_rows: 1695602
    })
    validation: Dataset({
        features: ['text', 'num_words'],
        num_rows: 3626
    })
})

In [17]:
print(bookcorpus_acc)
print(wikitext_acc)
print(bookcorpus_acc[0] / wikitext_acc[0])

[4579099811]
[98705551]
46.39151258068556
